## Bibliotecas

In [364]:
import pandas as pd
import numpy as np
import scipy as scp
from bs4 import BeautifulSoup
import requests as rq
import re
from requests.exceptions import HTTPError
import urllib
import urllib.request
from selenium import webdriver
from reppy.robots import Robots
from urllib.parse import urlparse
import time
import operator
import wget

## Crawler

In [365]:
Dominio = ['https://www.capefeargames.com/','https://www.mtgotraders.com/',
           'https://www.wizardscupboard.com/','https://abugames.com/',
           'https://www.cardkingdom.com/','http://www.starcitygames.com/',
           'https://www.tcgplayer.com/','https://scryfall.com/']

paginasVisitadas = []
Dados = []

In [366]:
# Função para marcar a page como visitada.
def Visitou (url):
    if url in paginasVisitadas:
        return True
    else:
        return False

In [367]:
# Função para verificar se a requisição sofreu algum problema.
def verificarRequest(url):
    response = rq.get(url)
    if response.status_code == 200:
        return True
    else:
        return False

In [368]:
def CrawlerDinâmico(url):
    #C:\\Users\\Usuário\\Downloads\\chromedriver76\\chromedriver.exe 
    driver = webdriver.Chrome("C:\\Users\\Usuário\\Downloads\\chromedriver76\\chromedriver.exe")
    driver.get(url)
    res = driver.execute_script("return document.documentElement.outerHTML")
    driver.quit
    return res

In [369]:
def verificarRobotTxt(url):
    robots = Robots.fetch(url + '/robots.txt')
    return robots.allowed(url, '*')

In [370]:
# Criar uma Tabela com as informações sobre links.
def Tabela(count,url,site):
    Dados.append(dict(zip(['Número','Link','Site'],[count,url,site])))

## Padrões

In [371]:
# Padroes links de Cartas

sentenca ='https://www.capefeargames.com/catalog/magic_singles-throne_of_eldraine/oko_thief_of_crowns/606112'
CapeFearGamesMagicCard = re.compile(r'(.)*(magic_singles)(.)+(\d)$') # score = 5 
matches = CapeFearGamesMagicCard.finditer(sentenca)
for match in matches:
    print(match)

sentenca2 = 'https://www.mtgotraders.com/store/M15_Accursed_Spirit.html'
MtGoTradersMagicCard = re.compile(r'(.)*(store)(.)+') # score 4 ou 5 (lista de links de cartas tbm é dessa forma)
matches2 = MtGoTradersMagicCard.finditer(sentenca2)
for match in matches2:
    print(match)

sentenca3 = 'https://www.wizardscupboard.com/academy-researchers-p-21746.html'    
WizardCupBoardMagicCard = re.compile(r'(.)*(\d)*(\.html)$')
matches3 = WizardCupBoardMagicCard.finditer(sentenca3)
for match in matches3:
    print(match)

sentenca4 = 'https://abugames.com/magic-the-gathering/singles/product-detail/Abundance/10th-Edition/English' 
AbuGamesMagicCard = re.compile(r'(.)*(magic-the-gathering)(.)+(singles)(.)(product-detail)(.)*(\w)$')
matches4 = AbuGamesMagicCard.finditer(sentenca4)
for match in matches4:
    print(match)

sentenca5 = 'https://www.cardkingdom.com/mtg/war-of-the-spark/zombie-army-token-008'
CardKingdomMagicCard = re.compile(r'(.)*(mtg)(.)*(\w)$')
matches5 = CardKingdomMagicCard.finditer(teste2)
for match in matches5:
    print(match)

sentenca6 = 'http://www.starcitygames.com/catalog/magic_the_gathering/product/404636/410829'
StarCityMagicCard = re.compile(r'(.)*(magic_the_gathering).(product)(.)*(\d)$')
matches6 = StarCityMagicCard.finditer(sentenca6)
for match in matches6:
    print(match)

sentenca7 = 'https://shop.tcgplayer.com/magic/core-set-2020/vampire-of-the-dire-moon?xid=pda7dbc375c11488d89c90a7adb114b70'
TcgPlayerMagicCard = re.compile(r'(.)*(magic)(.)*(\w)$')
matches7 =TcgPlayerMagicCard.finditer(sentenca7)
for match in matches7:
    print(match)

sentenca8 = 'https://scryfall.com/card/soi/262/shard-of-broken-glass' 
ScryfallMagicCard = re.compile(r'(.)*(card)(.)*(\d)(.)*$')
matches8 =ScryfallMagicCard.finditer(sentenca8)
for match in matches8:
    print(match)

padroesLinks = [CapeFearGamesMagicCard,CardKingdomMagicCard,
                TcgPlayerMagicCard,AbuGamesMagicCard,
                ScryfallMagicCard,StarCityMagicCard,
               WizardCupBoardMagicCard,MtGoTradersMagicCard]


if CardKingdomMagicCard.fullmatch(sentenca5):
    print('Sucesso!')
else:
    print('Falhou!')


if ScryfallMagicCard.fullmatch(sentenca3):
    print('Sucesso!')
else:
    print('Falhou!')

<re.Match object; span=(0, 97), match='https://www.capefeargames.com/catalog/magic_singl>
<re.Match object; span=(0, 58), match='https://www.mtgotraders.com/store/M15_Accursed_Sp>
<re.Match object; span=(0, 64), match='https://www.wizardscupboard.com/academy-researche>
<re.Match object; span=(0, 94), match='https://abugames.com/magic-the-gathering/singles/>
<re.Match object; span=(0, 70), match='https://www.cardkingdom.com/mtg/war-of-the-spark/>
<re.Match object; span=(0, 78), match='http://www.starcitygames.com/catalog/magic_the_ga>
<re.Match object; span=(0, 109), match='https://shop.tcgplayer.com/magic/core-set-2020/va>
<re.Match object; span=(0, 55), match='https://scryfall.com/card/soi/262/shard-of-broken>
Sucesso!
Falhou!


In [372]:
def Score(url):
    score = 0
    for padrao in padroesLinks:
        if padrao.fullmatch(url):
            score = score + 5
        else:
            score = score + 0
    return score

## * Heurística

In [399]:
def CardsCrawler(url,score):
    print('Dominio: ' + url.geturl())
    count = 0
    paginas = [(url,score)]
    # Ordenar pelo Score
    paginas.sort(key=operator.itemgetter(1),reverse=True)
    
    paginasVisitadas = []
    robots = Robots.fetch(url.geturl() + '/robots.txt')
    while count < 1000 and len(paginas) > 0:
        count = count + 1
        primeiraPagina = paginas.pop(0)
        print(count,'-',primeiraPagina[0].geturl(),' Qtd Paginas=', len(paginas))
       
        # Verifica se a pagina foi visitada.Se foi, procuro o proximo link da lista que não foi visitado.
        # Verifico se o hostname de cada pagina é igual ao do dominio, se não, rocuro o proximo link da lista (navegar apenas dentro do site)
        # Verifico se hostname existe , se não , procuro o proximo link que tenha.
        # Verifico se é possivel crawlear a pagina de acordo com as regras do robot.txt , se não , procuro o proximo link.
        # Verifico se a respota da requisição da pagina foi um sucesso(200) e retornou algo que a gente possa trabalhar se não pega o proximo da lista.
        while Visitou(primeiraPagina[0].geturl()) == True or primeiraPagina[0].hostname != url.hostname or primeiraPagina[0].hostname == None or robots.allowed(primeiraPagina[0].geturl(), '*') == False or verificarRequest(primeiraPagina[0].geturl()) != True:
            if len(paginas) == 0:
                print('lascou@!')
                return
            primeiraPagina = paginas.pop(0)
    
        # Verficar se o site é dinamico.
        if (Dominio[3] in url.geturl()) or (Dominio[6] in url.geturl()):
            texto = CrawlerDinâmico(primeiraPagina[0].geturl())
        else:
            #request - usando o content type
            codigo_fonte = rq.get(primeiraPagina[0].geturl(),headers={'content-type': 'text/html'})
            # texto do codigo_fonte.
            texto = codigo_fonte.text
        soup = BeautifulSoup(texto,'html.parser')

        # Encontrar todos os links da pagina(url).
        for link in soup.findAll('a'):
            href = link.get('href')
            # Correção de href relativos para absolutos.
            href = urllib.parse.urljoin(url.geturl(),href)                      
            paginas.append((urlparse(href),Score(href)))

        # Marco como visitada a urlparse    
        paginasVisitadas.append(primeiraPagina[0].geturl())
        
        #Salvar a pagina localmente.
        
        urllib.request.urlretrieve(primeiraPagina[0].geturl(), "Paginas/PaginasHeuristica/"+str(url.hostname)+str(count)+".txt")
        
        #esperar 300 milisegundos para visitar o prox link.
        time.sleep(.300)

        
        Tabela(count,primeiraPagina[0].geturl(),url.hostname)

In [400]:
CardsCrawler(urlparse(Dominio[1]),5)

Dominio: https://www.mtgotraders.com/
1 - https://www.mtgotraders.com/  Qtd Paginas= 0
2 - https://www.mtgotraders.com/store/bots.html  Qtd Paginas= 190
3 - https://www.mtgotraders.com/hotlist/  Qtd Paginas= 365
4 - https://www.mtgotraders.com/buylist/  Qtd Paginas= 539
5 - https://www.mtgotraders.com/store/papertomtgo.html  Qtd Paginas= 715
6 - http://puremtgo.com/articles/fun-fastbond  Qtd Paginas= 891
7 - https://twitter.com/MTGOTraders  Qtd Paginas= 1069
8 - https://www.mtgotraders.com/store/faq.html  Qtd Paginas= 1241
9 - https://www.mtgotraders.com/store/index.html#news  Qtd Paginas= 1417
10 - https://www.mtgotraders.com/hotlist/#/  Qtd Paginas= 1607
11 - https://www.mtgotraders.com/cgi-bin/shop-bin/sc/order.cgi?storeid=*1a4c945171a04aae08de729fc95b2a&function=show  Qtd Paginas= 1781
12 - https://www.mtgotraders.com/store/core_set_2020.html  Qtd Paginas= 1954
13 - https://www.mtgotraders.com/store/throne_of_eldraine.html  Qtd Paginas= 2489
14 - https://www.mtgotraders.com/store/w

109 - https://www.mtgotraders.com/store/Masters_Edition.html  Qtd Paginas= 47556
110 - https://www.mtgotraders.com/store/Masters_Edition_2.html  Qtd Paginas= 47996
111 - https://www.mtgotraders.com/store/Masters_Edition_3.html  Qtd Paginas= 48459
112 - https://www.mtgotraders.com/store/masters_edition_4.html  Qtd Paginas= 48923
113 - https://www.mtgotraders.com/store/7th_edition.html  Qtd Paginas= 49391
114 - https://www.mtgotraders.com/store/classic.html  Qtd Paginas= 49903
115 - https://www.mtgotraders.com/store/ravnica_mythic_edition.html  Qtd Paginas= 50260
116 - https://www.mtgotraders.com/store/amonkhet_invocations.html  Qtd Paginas= 50566
117 - https://www.mtgotraders.com/store/kaladesh_inventions.html  Qtd Paginas= 50959
118 - https://www.mtgotraders.com/store/zendikar_expeditions.html  Qtd Paginas= 51366
119 - https://www.mtgotraders.com/store/vintage_masters_original_p9.html  Qtd Paginas= 51719
120 - https://www.mtgotraders.com/store/ultimate_box_toppers.html  Qtd Paginas= 51

212 - https://www.mtgotraders.com/store/battle_for_zendikar.html  Qtd Paginas= 86134
213 - https://www.mtgotraders.com/store/oath_of_the_gatewatch.html  Qtd Paginas= 86608
214 - https://www.mtgotraders.com/store/khans_of_tarkir.html  Qtd Paginas= 87061
215 - https://www.mtgotraders.com/store/fate_reforged.html  Qtd Paginas= 87541
216 - https://www.mtgotraders.com/store/dragons_of_tarkir.html  Qtd Paginas= 88009
217 - https://www.mtgotraders.com/store/theros.html  Qtd Paginas= 88495
218 - https://www.mtgotraders.com/store/born_of_the_gods.html  Qtd Paginas= 88959
219 - https://www.mtgotraders.com/store/journey_into_nyx.html  Qtd Paginas= 89412
220 - https://www.mtgotraders.com/store/return_to_ravnica.html  Qtd Paginas= 89871
221 - https://www.mtgotraders.com/store/gatecrash.html  Qtd Paginas= 90336
222 - https://www.mtgotraders.com/store/dragons_maze.html  Qtd Paginas= 90804
223 - https://www.mtgotraders.com/store/innistrad.html  Qtd Paginas= 91247
224 - https://www.mtgotraders.com/stor

315 - https://www.mtgotraders.com/store/venser_vs_koth.html  Qtd Paginas= 131730
316 - https://www.mtgotraders.com/store/izzet_vs_golgari.html  Qtd Paginas= 132115
317 - https://www.mtgotraders.com/store/sorin_vs_tibalt.html  Qtd Paginas= 132505
318 - https://www.mtgotraders.com/store/heroes_vs_monsters.html  Qtd Paginas= 132873
319 - https://www.mtgotraders.com/store/jace_vs_vraska.html  Qtd Paginas= 133235
320 - https://www.mtgotraders.com/store/slivers.html  Qtd Paginas= 133621
321 - https://www.mtgotraders.com/store/fire_lightning.html  Qtd Paginas= 133923
322 - https://www.mtgotraders.com/store/graveborn.html  Qtd Paginas= 134213
323 - https://www.mtgotraders.com/store/From_the_Vault_Dragons.html  Qtd Paginas= 134499
324 - https://www.mtgotraders.com/store/From_the_Vault_Exiled.html  Qtd Paginas= 134728
325 - https://www.mtgotraders.com/store/from_the_vault_relics.html  Qtd Paginas= 134959
326 - https://www.mtgotraders.com/store/from_the_vault_legends.html  Qtd Paginas= 135192
327

418 - https://www.mtgotraders.com/store/betrayers_of_kamgawa.html  Qtd Paginas= 172004
419 - https://www.mtgotraders.com/store/saviors_of_kamigawa.html  Qtd Paginas= 172446
420 - https://www.mtgotraders.com/store/mirrodin.html  Qtd Paginas= 172881
421 - https://www.mtgotraders.com/store/darksteel.html  Qtd Paginas= 173355
422 - https://www.mtgotraders.com/store/5th_dawn.html  Qtd Paginas= 173803
423 - https://www.mtgotraders.com/store/ultimate_masters.html  Qtd Paginas= 174249
424 - https://www.mtgotraders.com/store/masters_25.html  Qtd Paginas= 174771
425 - https://www.mtgotraders.com/store/iconic_masters.html  Qtd Paginas= 175297
426 - https://www.mtgotraders.com/store/eternal_masters.html  Qtd Paginas= 175812
427 - https://www.mtgotraders.com/store/vintage_masters.html  Qtd Paginas= 176329
428 - https://www.mtgotraders.com/store/tempest_remastered.html  Qtd Paginas= 176864
429 - https://www.mtgotraders.com/store/onslaught.html  Qtd Paginas= 177317
430 - https://www.mtgotraders.com/s

521 - https://www.mtgotraders.com/store/index.html#news  Qtd Paginas= 210315
522 - https://www.mtgotraders.com/hotlist/#/  Qtd Paginas= 210505
523 - https://www.mtgotraders.com/cgi-bin/shop-bin/sc/order.cgi?storeid=*1a4c945171a04aae08de729fc95b2a&function=show  Qtd Paginas= 210679
524 - https://www.mtgotraders.com/store/core_set_2020.html  Qtd Paginas= 210852
525 - https://www.mtgotraders.com/store/throne_of_eldraine.html  Qtd Paginas= 211387
526 - https://www.mtgotraders.com/store/war_of_the_spark.html  Qtd Paginas= 211898
527 - https://www.mtgotraders.com/store/ravnica_allegiance.html  Qtd Paginas= 212386
528 - https://www.mtgotraders.com/store/guilds_of_ravnica.html  Qtd Paginas= 212878
529 - https://www.mtgotraders.com/store/modern_horizons.html  Qtd Paginas= 213378
530 - https://www.mtgotraders.com/store/modern_masters_2017.html  Qtd Paginas= 213885
531 - https://www.mtgotraders.com/store/modern_masters_2015.html  Qtd Paginas= 214397
532 - https://www.mtgotraders.com/store/modern_

624 - https://www.mtgotraders.com/store/masters_edition_4.html  Qtd Paginas= 257821
625 - https://www.mtgotraders.com/store/7th_edition.html  Qtd Paginas= 258289
626 - https://www.mtgotraders.com/store/classic.html  Qtd Paginas= 258801
627 - https://www.mtgotraders.com/store/ravnica_mythic_edition.html  Qtd Paginas= 259158
628 - https://www.mtgotraders.com/store/amonkhet_invocations.html  Qtd Paginas= 259464
629 - https://www.mtgotraders.com/store/kaladesh_inventions.html  Qtd Paginas= 259857
630 - https://www.mtgotraders.com/store/zendikar_expeditions.html  Qtd Paginas= 260264
631 - https://www.mtgotraders.com/store/vintage_masters_original_p9.html  Qtd Paginas= 260617
632 - https://www.mtgotraders.com/store/ultimate_box_toppers.html  Qtd Paginas= 260848
633 - https://www.mtgotraders.com/store/avatars.html  Qtd Paginas= 261257
634 - https://www.mtgotraders.com/store/Promos.html  Qtd Paginas= 261560
635 - https://www.mtgotraders.com/store/Theme_Deck_Set.html  Qtd Paginas= 262225
636 - 

725 - https://www.mtgotraders.com/store/oath_of_the_gatewatch.html  Qtd Paginas= 295513
726 - https://www.mtgotraders.com/store/khans_of_tarkir.html  Qtd Paginas= 295966
727 - https://www.mtgotraders.com/store/fate_reforged.html  Qtd Paginas= 296446
728 - https://www.mtgotraders.com/store/dragons_of_tarkir.html  Qtd Paginas= 296914
729 - https://www.mtgotraders.com/store/theros.html  Qtd Paginas= 297400
730 - https://www.mtgotraders.com/store/born_of_the_gods.html  Qtd Paginas= 297864
731 - https://www.mtgotraders.com/store/journey_into_nyx.html  Qtd Paginas= 298317
732 - https://www.mtgotraders.com/store/return_to_ravnica.html  Qtd Paginas= 298776
733 - https://www.mtgotraders.com/store/gatecrash.html  Qtd Paginas= 299241
734 - https://www.mtgotraders.com/store/dragons_maze.html  Qtd Paginas= 299709
735 - https://www.mtgotraders.com/store/innistrad.html  Qtd Paginas= 300152
736 - https://www.mtgotraders.com/store/dark_ascension.html  Qtd Paginas= 300607
737 - https://www.mtgotraders.c

828 - https://www.mtgotraders.com/store/izzet_vs_golgari.html  Qtd Paginas= 341020
829 - https://www.mtgotraders.com/store/sorin_vs_tibalt.html  Qtd Paginas= 341410
830 - https://www.mtgotraders.com/store/heroes_vs_monsters.html  Qtd Paginas= 341778
831 - https://www.mtgotraders.com/store/jace_vs_vraska.html  Qtd Paginas= 342140
832 - https://www.mtgotraders.com/store/slivers.html  Qtd Paginas= 342526
833 - https://www.mtgotraders.com/store/fire_lightning.html  Qtd Paginas= 342828
834 - https://www.mtgotraders.com/store/graveborn.html  Qtd Paginas= 343118
835 - https://www.mtgotraders.com/store/From_the_Vault_Dragons.html  Qtd Paginas= 343404
836 - https://www.mtgotraders.com/store/From_the_Vault_Exiled.html  Qtd Paginas= 343633
837 - https://www.mtgotraders.com/store/from_the_vault_relics.html  Qtd Paginas= 343864
838 - https://www.mtgotraders.com/store/from_the_vault_legends.html  Qtd Paginas= 344097
839 - https://www.mtgotraders.com/store/ftv_realms.html  Qtd Paginas= 344337
840 - h

931 - https://www.mtgotraders.com/store/guildpact.html  Qtd Paginas= 380256
932 - https://www.mtgotraders.com/store/dissension.html  Qtd Paginas= 380694
933 - https://www.mtgotraders.com/store/champions_of_kamigawa.html  Qtd Paginas= 381129
934 - https://www.mtgotraders.com/store/betrayers_of_kamgawa.html  Qtd Paginas= 381598
935 - https://www.mtgotraders.com/store/saviors_of_kamigawa.html  Qtd Paginas= 382040
936 - https://www.mtgotraders.com/store/mirrodin.html  Qtd Paginas= 382475
937 - https://www.mtgotraders.com/store/darksteel.html  Qtd Paginas= 382949
938 - https://www.mtgotraders.com/store/5th_dawn.html  Qtd Paginas= 383397
939 - https://www.mtgotraders.com/store/ultimate_masters.html  Qtd Paginas= 383843
940 - https://www.mtgotraders.com/store/masters_25.html  Qtd Paginas= 384365
941 - https://www.mtgotraders.com/store/iconic_masters.html  Qtd Paginas= 384891
942 - https://www.mtgotraders.com/store/eternal_masters.html  Qtd Paginas= 385406
943 - https://www.mtgotraders.com/sto